In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [3]:
train.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,4WKQSBB,1/2/2019,F,M,1987,1X1H,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
1,CP5S02H,1/6/2019,F,M,1981,UAOD,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2YKDILJ,1/6/2013,M,U,1991,748L,QZYX,90QI,0,0,...,0,0,0,0,0,0,0,0,0,1
3,2S9E81J,1/8/2019,M,M,1990,1X1H,BP09,56SI,0,0,...,0,0,0,1,0,0,0,0,0,0
4,BHDYVFT,1/8/2019,M,M,1990,748L,NO3L,T4MS,0,0,...,0,0,0,0,0,0,1,1,0,0


In [4]:
test.head()

,ID,join_date,sex,marital_status,birth_year,branch_code,occupation_code,occupation_category_code,P5DA,RIBP,...,AHXO,BSTQ,FM3X,K6QO,QBOL,JWFN,JZ9D,J9JW,GHYX,ECY3
0,F86J5PC,1/12/2018,M,M,1984,94KC,DZRV,90QI,0,0,...,0,0,0,0,0,0,0,0,0,0
1,H6141K3,1/10/2019,M,M,1996,1X1H,J9SY,90QI,0,0,...,0,0,0,1,0,0,0,0,0,0
2,RBAYUXZ,1/1/2020,F,W,1968,UAOD,2A7I,T4MS,0,0,...,0,0,0,1,0,0,0,0,0,0
3,KCBILBQ,1/2/2019,M,M,1989,94KC,2A7I,T4MS,0,0,...,0,0,0,0,0,0,0,0,0,0
4,LSEC1ZJ,1/2/2020,F,M,1982,UAOD,0KID,T4MS,0,0,...,0,0,0,0,0,0,1,0,0,0


In [5]:
len(train.columns) == len(test.columns)

True

In [32]:
train.iloc[:, 8:].sum(axis = 'columns').sort_values()

0         2
18162     2
18160     2
18159     2
18158     2
         ..
20488     8
27936     8
23995     8
28263     9
28367    14
Length: 29132, dtype: int64

In [30]:
train.iloc[:, 8:].sum()

P5DA       40
RIBP     1780
8NN1      157
7POT      316
66FJ      339
GYSR        4
SOP4      431
RVSZ    25328
PYUQ     2173
LJR9      354
N2MW      838
AHXO      539
BSTQ      324
FM3X      110
K6QO    21629
QBOL     6833
JWFN      311
JZ9D     1425
J9JW     1418
GHYX      902
ECY3     1102
dtype: int64

In [ ]:
print(train['sex'].unique())
print(test['sex'].unique())

In [ ]:
print(np.sort(train['marital_status'].unique()))
print(np.sort(test['marital_status'].unique()))

В тесте "f", а в треин "F". Заменим в тесте "f" на "F"

In [ ]:
print(np.sort(train['branch_code'].unique()))
print(np.sort(test['branch_code'].unique()))

In [ ]:
len(test['occupation_code'].unique()) -  np.sum(np.isin(test['occupation_code'].unique(), train['occupation_code'].unique()))

9 occupation_code из теста нет в треине

In [ ]:
len(test['occupation_category_code'].unique()) -  np.sum(np.isin(test['occupation_category_code'].unique(), train['occupation_category_code'].unique()))

In [ ]:
np.sum(np.isin(test['ID'].values, train['ID'].values))

Распределение таргета в различных группах

In [ ]:
def melting(data):

    target_cols = ['P5DA', 'RIBP', '8NN1',
           '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
           'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']

    data = pd.melt(data, id_vars = [col for col in data.columns if col not in target_cols], 
                    value_vars = target_cols,
                    value_name = 'demand')
    
    return data

In [ ]:
train_melted = melting(train)

In [ ]:
train_melted['variable'].value_counts()

In [ ]:
demand_counts = train_melted.groupby('variable').sum().sort_values('demand')
plt.figure(figsize = (10, 7))
_ = plt.bar(x = demand_counts.index, height = demand_counts['demand'])
_ = plt.xticks(rotation = 45)

In [ ]:
(-4*np.log(4/29132) - (29132-4)*np.log(1-(4/29132)))/29132

In [ ]:
train['GYSR'].sum()

Видно, что для каждого отдельного товара объем спроса на него отличается. Значит распределения спроса на товары разных категорий, скорее всего, различаются, следовательно можно попробовать строить отдельную модель для каждого товара

In [ ]:
def tranforming_date(data):
    data['join_date'] = pd.to_datetime(data['join_date'], dayfirst = True)
    data['timestamp'] = data['join_date'].apply(lambda x: x.timestamp())
    return data

In [ ]:
train = train.dropna()
train = tranforming_date(train)
test = test.dropna()
test = tranforming_date(test)

In [ ]:
train['timestamp'].max() > test['timestamp'].min()

Следовательно, пользователи из теста зарегестрированны не позже пользователей из треина. То есть нет смысла делать кросс валидацию на основе времени регестации пользователя

Распределения переменных

In [ ]:
train = train.dropna()
def tranforming_date(data):
    data['join_date'] = pd.to_datetime(data['join_date'], dayfirst = True)
    data['timestamp'] = data['join_date'].apply(lambda x: x.timestamp())
    return data

In [ ]:
train = tranforming_date(train)

In [ ]:
def melting(data):

    target_cols = ['P5DA', 'RIBP', '8NN1',
           '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
           'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']

    data = pd.melt(data, id_vars = [col for col in data.columns if col not in target_cols], 
                    value_vars = target_cols,
                    value_name = 'demand')
    
    return data

In [ ]:
train = melting(train)

In [ ]:
cat_features = ['sex', 'marital_status', 'branch_code', 'occupation_code', 'occupation_category_code']
def encoding(data, cat_features, encoder = LabelEncoder()):
    if encoder == 'One Hot':
        return pd.get_dummies(data, cat_features, drop_first = True)
    else:
        encoder = LabelEncoder()
        for column in cat_features:
            data[column] = encoder.fit_transform(data[column])
        return data

In [ ]:
train = encoding(train, cat_features)

In [ ]:
train = train.drop(['ID', 'join_date'], axis = 'columns')

In [ ]:
target_cols = ['P5DA', 'RIBP', '8NN1', '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 
               'N2MW', 'AHXO', 'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']

In [ ]:
for target_col in target_cols:
    _ = sns.pairplot(train[train['variable'] == target_col], hue = 'demand', corner = True)

target_cols = ['P5DA', 'RIBP', '8NN1',
       '7POT', '66FJ', 'GYSR', 'SOP4', 'RVSZ', 'PYUQ', 'LJR9', 'N2MW', 'AHXO',
       'BSTQ', 'FM3X', 'K6QO', 'QBOL', 'JWFN', 'JZ9D', 'J9JW', 'GHYX', 'ECY3']

train = pd.melt(train, id_vars = [col for col in train.columns if col not in target_cols], 
                value_vars = target_cols,
                value_name = 'demand')
train = pd.get_dummies(train, columns = ['marital_status', 'branch_code', 'occupation_category_code', 'variable', 'sex'])